# Introduction

Copyright

Distribution

In order to use the code the following libraries must be installed:
1. Pandas

In [191]:
import pandas as pd
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dwavebinarycsp


# Input and Preprocessing

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

### Input data file

In this section the following input is proceeded:
1. Input data file with information on ferry routes with
    - From: route start port
    - To: route end port
    - Distance: shortest direct distance between departure and arrival port

In [192]:
df_ports = pd.read_csv("sample_data_harbours_utf8.csv", sep = ";")
df_ports

,Column1,Bremerhaven,Brunsbüttel,Emden,Hamburg,Kiel
0,Bremerhaven,0,NaN,137.0,NaN,135.0
1,Brunsbüttel,81,0.0,NaN,36.0,54.0
2,Emden,137,NaN,0.0,NaN,NaN
3,Hamburg,117,36.0,NaN,0.0,90.0
4,Kiel,135,NaN,NaN,NaN,0.0


### Formatting of connection data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured to contain the information in an unstacked format. 

In [193]:
# rename 'Column1' to 'From'
df_ports=df_ports.rename(columns={'Column1':"From"})

# transform end harbours in columns to rows
df_routes= pd.melt(df_ports,id_vars=['From'],var_name="To",value_name='Distance')

# filter NaN-values
df_routes=df_routes[df_routes["Distance"].notnull()]

# filter out 0.0 values
df_routes=df_routes[df_routes.Distance != 0]

#reset index
df_routes=df_routes.reset_index(drop=True)
df_routes

,From,To,Distance
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Hamburg,Brunsbüttel,36.0
5,Bremerhaven,Emden,137.0
6,Brunsbüttel,Hamburg,36.0
7,Bremerhaven,Kiel,135.0
8,Brunsbüttel,Kiel,54.0
9,Hamburg,Kiel,90.0


### User input and validation of departure and destination harbour 

In this section the manual input is requested and validated by the following criteria:
1. departure and destination harbour are part of the input 

If input is invalid, the according error message is displayed.

In [194]:
#Check whether the departure harbour is element of the input data
while True:
    #Entering departure harbour
    departure=input(str("Please enter departure port:"))
    if not departure in df_routes['From'].values:
        print("Entered port is not defined as a depature port in the input file. Please enter valid departure port.")
        continue
    else:
        break

#Check whether the destination harbour is element of the input data
while True:
    #Entering destination harbour
    destination=input(str("Please enter destination port:"))
    if not destination in df_routes['To'].values:
        print("Entered port is not defined as a destination port in the input file. Please enter destination valid port.")
        continue
    if destination == departure:
        print("Entered destination port is equal to departure port. Please enter valid destination port.")
        continue
    else:
        #we're happy with the value given.
        #we're ready to exit the loop.
        break


### Data enhancement of connection data

A problem can be formulated in different ways in order to provide the quantum computer with it. In this example, the problem is formulated as a binary quadratic model (BQM) in the form of a QUBO. The mathematical phrase can be attained by a support class called Constraint Satisfaction Problem (CSP) or can directly be input. In this case, the problem will first be represented by a CSP. The CSP constitutes of 
1. all variables and 
2. constraints on the variables. 

As the variables in this example will represent the routes between two ports a set of support variables is needed in order to represent the entered departure and destination port. Therefor, artificial routes from "start" port to each real port and to "end" port from each real port are created in the following code. 


In [195]:
df_starts = pd.DataFrame({'From':'start', 'To':df_routes['From'].unique(),'Distance':0})
df_ends = pd.DataFrame({'From':df_routes['To'].unique(), 'To':'end','Distance':0})
df_routes= pd.concat([df_starts,df_routes,df_ends])
df_routes

,From,To,Distance
0,start,Brunsbüttel,0.0
1,start,Emden,0.0
2,start,Hamburg,0.0
3,start,Kiel,0.0
4,start,Bremerhaven,0.0
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Hamburg,Brunsbüttel,36.0


# Data Preparation for Quantum Computing (Definition of Classes)

In this section, all methods are defined that are later used to
1. create Contraint Satisfaction Problem (CSP)
2. formulate Binary Quadratic Model (BQM) as QUBO. 

### Basic functions

In [213]:
def sum_to_two_or_zero(*args):
        """Checks to see if the args sum to either 0 or 2.
        """
        sum_value = sum(args)
        return sum_value in [0, 2]
        
def sum_smaller_equal_one(*args):
        """Checks to see if the args sum to either smaller or equal to 1.
        """
        sum_value = sum(args)
        return sum_value in [0, 1]

### Definition class Ferry

In [216]:
class Ferryarea:
    def __init__(self):
        """ Initializes object of Type Ferryarea"""
        #assign data to object Ferryarea
        self.df_routes = df_routes#df_routes.values.tolist()
        self.ports = pd.DataFrame({'ports':np.append(df_routes['To'].unique(),df_routes['From'].unique())}).drop_duplicates().values.tolist()
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        self.n=0
        
#brainstorming
# - every route can be 0 or 1
# - every port can only be approached once
# - only one route to and from a harbour 
#
#- idea: routes = variables
# - sum of the routes to one port + sum of the routes from one port = 0 OR 2
# - how to make sure that only max one way is chosen? sum of routes to one port <= 1 and routes from one port <=1
# - makes 3 constraints per port

#list of all ports: self.ports
#dataframe of all routes
#target format: dataframe with according port only
    #list format: [['Brunsbüttel', 'Bremerhaven', 81.0], ['Emden', 'Bremerhaven', 137.0], ['Hamburg',       'Bremerhaven', 117.0], ['Kiel', 'Bremerhaven', 135.0], ['Wismar', 'Bremerhaven', 585.0], ['Rotterdam', 'Bremerhaven', 255.0], ['Bremerhaven', 'Emden', 137.0], ['Bremerhaven', 'Kiel', 135.0], ['Bremerhaven', 'Aberdeen', 435.0]]
    #target: dictionary "BB-BH-81"
#every line in dataframe -> one variable in dictionary

    def get_bqm(self):
        for port in self.ports:
                #print("port", port[0])
            self.n=0

            #asign to directions Dataframe a Dataframe containing only direction to and from the port
            df_directions_from= self.df_routes[self.df_routes['From']==port[0]]
            df_directions_to= self.df_routes[self.df_routes['To']==port[0]]
            df_directions= pd.concat([df_directions_to,df_directions_from])
            
            #creates sets of labels as basis for constraint defintion
            directions_all = set((df_directions['From']+'-'+df_directions['To']+'-'+df_directions['Distance'].astype(str)).values.tolist())
            directions_to= set((df_directions_to['From']+'-'+df_directions_to['To']+'-'+df_directions_to['Distance'].astype(str)).values.tolist())
            directions_from= set((df_directions_from['From']+'-'+df_directions_from['To']+'-'+df_directions_from['Distance'].astype(str)).values.tolist())
            
            #create dictionary with variable names referring to the naming convention "Start-End-Distance"
                #directions=set(labels_all.values.tolist())
            print("all:", directions_all)
            print("to:", directions_to)
            print('from', directions_from)
            
            #add constraint sum_to_two_or_zero
            self.csp.add_constraint(sum_to_two_or_zero,directions_all)

            #add constraint 'To' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one,directions_to)
            #add constraint 'From' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one, directions_from)
            
        print(self.csp.constraints)



### Initialization of Ferryarea

In [215]:
f=Ferryarea()
bqm= f.get_bqm()

Directions: {'Brunsbüttel-Hamburg-36.0', 'Hamburg-Brunsbüttel-36.0', 'Brunsbüttel-end-0.0', 'start-Brunsbüttel-0.0', 'Brunsbüttel-Bremerhaven-81.0', 'Brunsbüttel-Kiel-54.0'}
type: <class 'set'>
Directions: {'start-Emden-0.0', 'Bremerhaven-Emden-137.0', 'Emden-end-0.0', 'Emden-Bremerhaven-137.0'}
type: <class 'set'>
Directions: {'Brunsbüttel-Hamburg-36.0', 'start-Hamburg-0.0', 'Hamburg-Kiel-90.0', 'Hamburg-Brunsbüttel-36.0', 'Hamburg-end-0.0', 'Hamburg-Bremerhaven-117.0'}
type: <class 'set'>
Directions: {'Hamburg-Kiel-90.0', 'start-Kiel-0.0', 'Bremerhaven-Kiel-135.0', 'Kiel-Bremerhaven-135.0', 'Kiel-end-0.0', 'Brunsbüttel-Kiel-54.0'}
type: <class 'set'>
Directions: {'Bremerhaven-Kiel-135.0', 'Kiel-Bremerhaven-135.0', 'Bremerhaven-Emden-137.0', 'Bremerhaven-end-0.0', 'start-Bremerhaven-0.0', 'Brunsbüttel-Bremerhaven-81.0', 'Hamburg-Bremerhaven-117.0', 'Emden-Bremerhaven-137.0'}
type: <class 'set'>
Directions: {'Brunsbüttel-end-0.0', 'Emden-end-0.0', 'Hamburg-end-0.0', 'Kiel-end-0.0', 'Br

# Creation of BQM and Transfer to Quantum Sampler